In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import time

start_time = time.time()

# Data loading optimization
def get_data_loaders(batch_size=2048, num_workers=4, pin_memory=True):
    
    # Randomly shift imate up to 10% or rotate up to 10 degrees    
    # Normalizes the data using mean=0.1307 and std=0.3081
    train_transforms = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = MNIST('data', train=True, transform=train_transforms, download=True)
    test_dataset = MNIST('data', train=False, transform=test_transforms, download=True)

    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers, # Nr of processes to use for data loading
        pin_memory=pin_memory # Copy Tensors to CUDA pinned memory for faster data transfer to GPU
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        num_workers=num_workers,
        pin_memory=pin_memory
    )
    
    return train_loader, test_loader

class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.flatten = nn.Flatten()
        
        self.fc_layers = nn.Sequential( # Fully connected layers
            nn.Linear(28 * 28, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc_layers(x)
        return x

# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MNISTNet().to(device)

# Move model to GPU before creating optimizer
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Cut learning rate by half if no test accuracy improvement over 3 epochs
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

# Get data loaders with optimized settings
train_loader, test_loader = get_data_loaders()

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        # Move images and labels to GPU, asynchronously if possible
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)  # Slightly more efficient than zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1) # Find the prediction value(index) for each sample(row)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(train_loader), 100. * correct / total

@torch.no_grad()  # More efficient than with torch.no_grad():
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        # Move images and labels to GPU asynchronously if possible
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(images)
        _, predicted = outputs.max(1) # Find the prediction value(index) for each sample(row)
        total += labels.size(0) 
        # Count the number of instances correctly predicted
        correct += predicted.eq(labels).sum().item()
    
    return 100. * correct / total

# Training loop
n_epochs = 100
best_acc = 0

for epoch in range(n_epochs):
    epoch_start = time.time()
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    test_acc = evaluate(model, test_loader)
    
    if test_acc > best_acc:
        best_acc = test_acc
    
    scheduler.step(test_acc)
    
    print(f'Epoch: {epoch+1}/{n_epochs}')
    print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
    print(f'Test Acc: {test_acc:.2f}% | Best Acc: {best_acc:.2f}%')
    print(f'Epoch Time: {time.time() - epoch_start:.2f}s')
    print('-' * 50)
    
print(f'Total Running Time: {time.time() - start_time:.2f}s')

Epoch: 1/100
Train Loss: 1.1850 | Train Acc: 60.37%
Test Acc: 90.97% | Best Acc: 90.97%
Epoch Time: 24.03s
--------------------------------------------------
Epoch: 2/100
Train Loss: 0.4419 | Train Acc: 85.91%
Test Acc: 95.13% | Best Acc: 95.13%
Epoch Time: 23.80s
--------------------------------------------------
Epoch: 3/100
Train Loss: 0.2850 | Train Acc: 91.26%
Test Acc: 96.18% | Best Acc: 96.18%
Epoch Time: 22.68s
--------------------------------------------------
Epoch: 4/100
Train Loss: 0.2262 | Train Acc: 92.96%
Test Acc: 97.05% | Best Acc: 97.05%
Epoch Time: 22.55s
--------------------------------------------------
Epoch: 5/100
Train Loss: 0.1966 | Train Acc: 93.94%
Test Acc: 97.62% | Best Acc: 97.62%
Epoch Time: 22.40s
--------------------------------------------------
Epoch: 6/100
Train Loss: 0.1702 | Train Acc: 94.74%
Test Acc: 97.79% | Best Acc: 97.79%
Epoch Time: 23.20s
--------------------------------------------------
Epoch: 7/100
Train Loss: 0.1587 | Train Acc: 95.11%


In [8]:
# Save the trained model
torch.save(model.state_dict(), 'mnist_model.pth')

# Generate and save predictions
@torch.no_grad()
def generate_predictions(model, test_loader):
    model.eval()
    all_predictions = []
    
    for images, _ in test_loader:
        images = images.to(device, non_blocking=True)
        outputs = model(images)
        _, predicted = outputs.max(1)
        all_predictions.extend(predicted.cpu().numpy())
    
    return all_predictions

# Generate predictions
predictions = generate_predictions(model, test_loader)

# Create submission file
import pandas as pd

submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'target': predictions
})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("Model saved as 'mnist_model.pth'")
print("Predictions saved as 'submission.csv'")

Model saved as 'mnist_model.pth'
Predictions saved as 'submission.csv'
